In [6]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import fairnessMetrics as fm

In [7]:
def readImages(folderName, image_size, batch_size):
    train_ds = keras.preprocessing.image_dataset_from_directory(
        folderName,
        validation_split=0.2,
        subset="training",
        seed=1337,
        image_size=image_size,
        batch_size=batch_size
    )
    val_ds = keras.preprocessing.image_dataset_from_directory(
        folderName,
        validation_split=0.2,
        subset="validation",
        seed=1337,
        image_size=image_size,
        batch_size=batch_size
    )
    return (train_ds, val_ds)

In [8]:

def build_model():
    inputs = layers.Input(shape=(224, 224, 3))
    x = inputs
    model = keras.applications.EfficientNetV2B0(include_top=False, input_tensor=x, weights="imagenet")

    #model.trainable = False

    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(1, activation="sigmoid", name="pred")(x)

    model = keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = keras.optimizers.Adam(learning_rate=1e-3)
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy",
                                                                  tf.keras.metrics.TruePositives(), 
                                                                  tf.keras.metrics.TrueNegatives(),
                                                                  tf.keras.metrics.FalsePositives(), 
                                                                  tf.keras.metrics.FalseNegatives(), 
                                                                  fm.TruePositiveRate(),
                                                                  fm.TrueNegativeRate(),
                                                                  fm.FalsePositiveRate(),
                                                                  fm.FalseNegativeRate(),
                                                                  fm.PositivePredictedValue(),
                                                                  fm.FalseDiscoveryRate(),
                                                                  fm.NegativePredictedValue(),
                                                                  fm.FalseOmissionRate(),
                                                                  fm.BinaryDemographicParityDiff(),
                                                                  fm.DemographicParity(),
                                                                  fm.BinaryEqualizedOddsDiff(),
                                                                  fm.BinaryProportionalParityDiff(),
                                                                  fm.ProportionalParity(),
                                                                  fm.BinaryPredictiveRateParityDiff(),
                                                                  fm.PredictiveRateParity(),
                                                                  fm.BinaryAccuracyParityDiff(),
                                                                  fm.AccuracyParity(),
                                                                  fm.BinaryFalseNegativeRateParityDiff(),
                                                                  fm.BinaryFalsePositiveRateParityDiff(),
                                                                  fm.BinaryNegativePredictiveRateParityDiff(),
                                                                  fm.NegativePredictiveRateParity(),
                                                                  fm.BinarySpecificityParityDiff()]
    )
    return model


In [9]:
def train_model(model, epochs, data):
    model.fit(data[0], epochs=epochs, validation_data=data[1])

In [10]:
image_size = (224, 224)
batch_size = 32
data = readImages("../../FairFace", image_size, batch_size)
#tf.config.run_functions_eagerly(True)


input_shape = (image_size[0], image_size[1], 3)
model = build_model()
train_model(model, 40, data)

Found 97698 files belonging to 2 classes.
Using 78159 files for training.
Found 97698 files belonging to 2 classes.
Using 19539 files for validation.
Epoch 1/40
   9/2443 [..............................] - ETA: 1:48:07 - loss: 0.8092 - accuracy: 0.6007 - true_positives_21: 94.0000 - true_negatives_14: 79.0000 - false_positives_17: 57.0000 - false_negatives_18: 58.0000 - true_positive_rate: 0.6184 - true_negative_rate: 0.5809 - false_positive_rate: 0.4191 - false_negative_rate: 0.3816 - positive_predicted_value: 0.6225 - false_discovery_rate: 0.3775 - negative_predicted_value: 0.5766 - false_omission_rate: 0.4234 - binary_demographic_parity_diff: 14.0000 - demographic_parity: 151.0000 - binary_equalized_odds_diff: -0.0375 - binary_proportional_parity_diff: 0.0486 - proportional_parity: 0.5243 - binary_predictive_rate_parity_diff: 0.0459 - predictive_rate_parity: 0.6225 - binary_accuracy_parity_diff: 0.2014 - accuracy_parity: 0.6007 - false_negative_rate_parity_diff: 0.0375 - false_posit

KeyboardInterrupt: 